In [1]:
RDATE = 20210131

import numpy as np
import pandas as pd
import pandasql
import sqlite3

pd.set_option("display.max_columns", None) 
pd.set_option("display.max_colwidth", 1000) #huruf dlm column
pd.set_option("display.max_rows", 100)
pd.set_option("display.precision", 2) #2 titik perpuluhan

#/*TO IMPORT CRRS FILE*/
CRRS = pd.read_excel(r"T:\MIB Risk Management\Credit Risk Analytics - Historical\02_Data_Source\11_CRRS\\"+str(RDATE)[:4]+"\\"+str(RDATE)+"\Report_Corp_CRDM_"+str(RDATE)+".xlsx")


#/*TO IMPORT FRS FILE*/
FRS = pd.read_excel(r"T:\\MIB Risk Management\\Credit Risk Analytics - Historical\\02_Data_Source\\01_Corporate_Financing\\01_Portfolio\\"+str(RDATE)+"\\GB Monthly Loan Outstanding "+str(RDATE)+".xlsx")


#/*TO IMPORT ASSET QUALITY FILE*/
#/*TO IMPORT IMPAIRED ACCOUNTS*/
IMPAIRED = pd.read_excel(r"T:\\MIB Risk Management\\Credit Risk Analytics - Historical\\02_Data_Source\\01_Corporate_Financing\\02_Asset_Quality\\"+str(RDATE)+"\\Asset_Quality_Master_List_"+str(RDATE)+".xlsx", sheet_name="Impaired_Accounts")


#/*TO IMPORT SMA*/
SMA = pd.read_excel(r"T:\\MIB Risk Management\\Credit Risk Analytics - Historical\\02_Data_Source\\01_Corporate_Financing\\02_Asset_Quality\\"+str(RDATE)+"\\Asset_Quality_Master_List_"+str(RDATE)+".xlsx", sheet_name="SMA")


#/*TO IMPORT WATCHLIST*/
WATCHLIST = pd.read_excel(r"T:\\MIB Risk Management\\Credit Risk Analytics - Historical\\02_Data_Source\\01_Corporate_Financing\\02_Asset_Quality\\"+str(RDATE)+"\\Asset_Quality_Master_List_"+str(RDATE)+".xlsx", sheet_name="Watchlist")


#/*TO IMPORT EWS*/
EWS = pd.read_excel(r"T:\\MIB Risk Management\\Credit Risk Analytics - Historical\\02_Data_Source\\01_Corporate_Financing\\02_Asset_Quality\\"+str(RDATE)+"\\Asset_Quality_Master_List_"+str(RDATE)+".xlsx", sheet_name="EWS")


#/*TO IMPORT R&R*/
RR = pd.read_excel(r"T:\\MIB Risk Management\\Credit Risk Analytics - Historical\\02_Data_Source\\01_Corporate_Financing\\02_Asset_Quality\\"+str(RDATE)+"\\Asset_Quality_Master_List_"+str(RDATE)+".xlsx", sheet_name="R&R")


#/*TO IMPORT RDMS*/ /*new format of RDMS dataset 26 Nov 2019*/
RDMS = pd.read_excel(r"T:\\MIB Risk Management\\Credit Risk Analytics - Historical\\02_Data_Source\\12_RDMS\\"+str(RDATE)+"\\RDMS_CB_"+str(RDATE)+".xlsx")

#/*User Defined*/
RATING_CATEGORY = pd.read_excel(r"T:\MIB Risk Management\Credit Risk Analytics - Historical\03_Data_Warehouse\01_Data_Dictionary\Rating_Category.xls")


SUB_MARKET_SEGMENT = pd.read_excel(r"T:\MIB Risk Management\Credit Risk Analytics - Historical\03_Data_Warehouse\01_Data_Dictionary\Sub Market Segment Code.xls")


In [11]:

RR001 = RR.iloc[np.where(RR.Credit_Facility_O_S_Mil != 0)].sort_values(by=['Reporting_Month','CUS_NO'],ascending=[False,True]) 

W001 = WATCHLIST.sort_values(by=['Reporting_Month','CUS_NO'],ascending=[False,True]) 

#/*EWS - ADDED 17 OCT 2018*/
EWS001 = EWS.sort_values(by=['Reporting_Month','CUS_NO'],ascending=[False,True]) 

#/*+AVOID DUPLICATION*/
SMA.O_S__RM_MILLION_.fillna(0,inplace=True)
SMA.Reporting_Month.fillna('NA',inplace=True)
SMA.CUS_NO.fillna('NA',inplace=True)

S001 = SMA.groupby(['Reporting_Month','CUS_NO'])[['O_S__RM_MILLION_']].sum().reset_index().sort_values(by=['Reporting_Month','CUS_NO'],ascending=[False,True])

#/*SMA IS AT ACCOUNT-LEVEL*/
SMA.columns = SMA.columns.str.replace(" ", "_")

S002_1 = SMA.sort_values(by=['Reporting_Month','ACCOUNT_NUMBER'],ascending=[False,True]) 

#sbb 	2013-06-30	SMA	3871044	SYARIKAT BORCOS SHIPPING SDN BHD	461127800761, 461127800545	I	100.41	TL interest and principal repayment in arrears for 2 months.	NaN	NaN	NaN	NaN	NaN	NaN	461127800761, 461127	461127800761	461127800545
b = S002_1.assign(ACCOUNT_NUMBER=S002_1.ACCOUNT_NUMBER.str.split(','))
S002 = b.explode('ACCOUNT_NUMBER')
#cek
#SMA.iloc[np.where((SMA.BORROWER == 'SYARIKAT BORCOS SHIPPING SDN BHD')&(SMA.Reporting_Month=='2013-06-30'))]
#S002.iloc[np.where((S002.BORROWER == 'SYARIKAT BORCOS SHIPPING SDN BHD')&(S002.Reporting_Month=='2013-06-30'))]

S002['ACCOUNT_NO'] = S002.ACCOUNT_NUMBER.str[:20]

I001 = IMPAIRED.sort_values(by=['Reporting_Month','CUS_NO'],ascending=[False,True]) 

I002 = I001[['CUS_NO','NCNPL_Flag']].drop_duplicates()

C001 = CRRS.sort_values(by=['GCIF_NUM'],ascending=[True]) 
C001['CUS_NO'] = C001.GCIF_NUM.astype(str).str.strip()

RDMS001 = RDMS

RDMS001['CUS_NO'] = RDMS001.COUNTERPARTY_ORIGINAL.astype(str).str.strip()

def RATING1(x, y):
    if x == "PSE(SA)":
        return "PSE"
    elif x == "SOV(SA)":
        return "SOVEREIGN"
    elif x == "SOV(SA)":
        return "UNRATED"
    else:
        return y

RDMS001['RATING1'] = RDMS001.apply(lambda x: RATING1(x['CUSTOMER_FIELD19'], x['INTERNAL_RATING_ORIGINAL']), axis=1)

RDMS001.EAD.fillna(0,inplace=True)
RDMS001.RWA.fillna(0,inplace=True)
RDMS001.CUS_NO.fillna('NA',inplace=True)
RDMS001.RATING1.fillna('NA',inplace=True)

RDMS002 = RDMS001.iloc[np.where(RDMS.DEALBOOK.isin(["MIB_OTH","MIB_RA"]))].groupby(['CUS_NO','RATING1'])[['EAD','RWA']].sum().reset_index().rename(columns = {'RATING1':'RATING','EAD':'EAD_AMOUNT','RWA':'RWAA_AFTER_SCALING_FACTOR'})

RDMS003 = RDMS002.sort_values(by=['CUS_NO','EAD_AMOUNT','RWAA_AFTER_SCALING_FACTOR'],ascending=[True,False,False]).drop_duplicates()


FRS.columns = FRS.columns.str.replace(" ", "_")
B001 = FRS.sort_values(by=['M_Financing_Type_Cd','M_Cus_No'],ascending=[True,True])
B001['CUS_NO'] = B001.M_Cus_No.astype(str).str.strip()

B001_1 = B001.sort_values(by=['M_Account_No'],ascending=[True])

B001_2 = B001_1.merge(S002[['ACCOUNT_NUMBER','O_S__RM_MILLION_']].rename(columns = {'O_S__RM_MILLION_':'SMA_CUS_BAL','ACCOUNT_NUMBER':'M_Account_No'}),on = 'M_Account_No' , how='left').sort_values(by=['CUS_NO'],ascending=[True])

B001_2.CUS_NO = B001_2.CUS_NO.astype(str)
W001.CUS_NO = W001.CUS_NO.astype(str)
S001.CUS_NO = S001.CUS_NO.astype(str)
EWS001.CUS_NO = EWS001.CUS_NO.astype(str)
I002.CUS_NO = I002.CUS_NO.astype(str)
RR001.CUS_NO = RR001.CUS_NO.astype(str)
C001.CUS_NO = C001.CUS_NO.astype(str)
RDMS003.CUS_NO = RDMS003.CUS_NO.astype(str)

W001['WATCHLIST_FLAG'] = 1
S001['SMA_FLAG'] = 1
I002['IMPAIRED_FLAG'] = 1
EWS001['EWS_FLAG'] = 1

W001.ACCOUNT_DETAILS__RM_MILLION__O_S.fillna(0, inplace=True)

B002 = B001_2.merge(W001[['CUS_NO','ACCOUNT_DETAILS__RM_MILLION__O_S','WATCHLIST_FLAG']].rename(columns = {'ACCOUNT_DETAILS__RM_MILLION__O_S':'WATCHL_BAL'}),on='CUS_NO',how='left').merge(S001[['CUS_NO','SMA_FLAG']],on='CUS_NO',how='left').merge(EWS001[['CUS_NO','EWS_FLAG']],on='CUS_NO',how='left').merge(I002[['CUS_NO','NCNPL_Flag','IMPAIRED_FLAG']],on='CUS_NO',how='left').merge(RR001[['CUS_NO','AQ_Quality_Status']].rename(columns = {'AQ_Quality_Status':'RR_TYPE'}),on='CUS_NO',how='left').merge(C001[['CUS_NO','RDMS_ADJ_BRR','FIN_FRR']],on='CUS_NO',how='left').merge(RDMS003[['CUS_NO','RATING']].rename(columns = {'RATING':'RDMS_RATING'}),on='CUS_NO',how='left')

B002.WATCHLIST_FLAG.fillna(0, inplace=True)
B002.SMA_FLAG.fillna(0, inplace=True)
B002.IMPAIRED_FLAG.fillna(0, inplace=True)
B002.EWS_FLAG.fillna(0, inplace=True)


def NEWLY_IF_FLAG(x):
    if x == 'Y':
        return 1
    else:
        return 0
    
B002['NEWLY_IF_FLAG'] = B002.NCNPL_Flag.apply(NEWLY_IF_FLAG)

B002.loc[B002.RR_TYPE == 'Rescheduled','RESCHEDULED_FLAG'] = 1
B002.loc[B002.RR_TYPE == 'Restructured','RESTRUCTURED_FLAG'] = 1
B002.loc[B002.RR_TYPE.isin(['Rescheduled','Restructured']),'R_N_R_FLAG'] = 1


B002.loc[B002.RDMS_ADJ_BRR.isna(), 'CRRS_RATING'] = "NA"
B002.loc[~(B002.RDMS_ADJ_BRR.isna()), 'CRRS_RATING'] = B002.RDMS_ADJ_BRR.str.strip()

B002['RATING'] = B002.RDMS_RATING.str.strip()
B002.loc[B002.RATING.isna(),'RATING'] = B002.CRRS_RATING.str.strip()
B002.loc[(B002.RATING.isna())|(B002.RATING=='NA'),'RATING'] = 'UNRATED'
B002.loc[B002.RATING == "DEFAULTED",'RATING'] = B002.CRRS_RATING.str.strip()

B002['COMBINED_RATING'] = B002.RATING

#/*JUST ADDED 14 AUG 2018. STARTING JUNE 2018, WE USE MISC CODE*/
#/*FORMAT ISC $170.;   JUST ADDED 14 AUG 2018. STARTING JUNE 2018, WE USE MISC CODE*/

B002.M_Msic_Code = B002.M_Msic_Code.str.upper()
B002.M_Msic_Code = B002.M_Msic_Code.str.strip()

B002.loc[B002.M_Msic_Code.isin(['01120','0113X','01140','0119XA','0119XB','01261A','01261B','01262A','01262B','01263','01269','01273','01279','01291','01299','0130X','0141X','0144X','0145X','0146XA','0146XB','0146XC','0149X','01500','016XX','0170X','0210X','0220X','0230X','0240X','031XXA','031XXB','031XXC','031XXD','032XXA','032XXB','032XXC', '1120','113X','1140','119XA','119XB','1261A','1261B','1262A','1262B','1263','1269','1273','1279','1291','1299','130X','141X','144X','145X','146XA','146XB','146XC','149X','1500','16XX','170X','210X','220X','230X','240X','31XXA','31XXB','31XXC','31XXD','32XXA','32XXB','32XXC']),'ISC_PARENT'] = "01. Agriculture, Forestry and Fishing"
B002.loc[B002.M_Msic_Code.isin(['05XXX','0610X','0620X','0710X','07210','07291','07299','08XXX','0910XA','0910XB','0910XC','0910XD','0910XE','0910XF','0910XG','09900', '5XXX','610X','620X','710X','7210','7291','7299','8XXX','910XA','910XB','910XC','910XD','910XE','910XF','910XG','9900']),'ISC_PARENT'] = "02. Mining and Quarrying"
B002.loc[B002.M_Msic_Code.isin(['1010X','1020X','1030X','10401A','10401B','10402A','10402B','10499','1050X','106XX','107XX','10800','110XXA','110XXB','12000','13XXX','14XXX','151XX','1520X','16XXX','1701X','181XX','18200','19XXXA','19XXXB','20XXX','21XXX','22XXXA','22XXXB','22XXXC','22XXXD','22XXXE','22XXXF','22XXXG','23XXXA','23XXXB','23XXXC','2410XA','2410XB','24201','24202','24209A','24209B','2431XA','2431XB','24320','251XX','25200','25910','25920','25930','2599X','26300','265XX','2670X','26999','27500','27999','28170','2819XA','2819XB','2819XC','2819XD','2819XE','28220','28299A','28299B','28299C','28299D','28299E','29XXXA','29XXXB','29XXXC','301XXA','301XXB','301XXC','301XXD','30200','30999A','30999B','30999C','3100X','32500','3290X','33XXXA','33XXXB','33XXXC','33XXXD','33XXXE','33XXXF','33XXXG','33XXXH']),'ISC_PARENT'] = "03. Manufacturing"
B002.loc[B002.M_Msic_Code.isin(['3510XA','3510XB','3510XC','3510XD','3510XE','3510XF','3510XG','3520XA','3520XB','3530X']),'ISC_PARENT'] = "04. Electricity, Gas, Steam and Air Conditioning Supply"
B002.loc[B002.M_Msic_Code.isin(['360XX','3700X','38112','38199','382XXA','382XXB','382XXC','3830X','39000']),'ISC_PARENT'] = "05. Water Supply; Sewerage, Waste Management and Remediation Activities"
B002.loc[B002.M_Msic_Code.isin(['41001A','41001B','41001C','41001D','41001E','41001F','41001G','41001H','41002A','41002B','41002C','41002D','41002E','41002F','41002G','41002H','41002I','41002J','41003','41009','4210XA','4210XB','4210XC','4210XD','4210XE','4210XF','4210XG','4220XA','4220XB','4220XC','4220XD','429XXA','429XXB','429XXC','431XX','432XX','4330X','4390X']),'ISC_PARENT'] = "06. Construction"
B002.loc[B002.M_Msic_Code.isin(['451XXA','451XXB','451XXC','451XXD','451XXE','4520X','4530XA','4530XB','4540X','462XXA','462XXB','462XXC','463XXA','463XXB','463XXC','463XXD','4641X','46496','46499','46510','46595','46599A','46599B','46599C','46999','4711X','4719X','472XXA','472XXB','472XXC','472XXD','478XX','479XX','40799A','40799B','40799C','40799D','40799E']),'ISC_PARENT'] = "07. Wholesale and Retail Trade; Repair of Motor Vehicles and Motorcycles"
B002.loc[B002.M_Msic_Code.isin(['4911X','4912X','4921X','4922X','49230','49300','5011X','50121','50122','5021X','5022X','51101A','51101B','51102','51103','51201','51202','51203','5210XA','5210XB','5221X','5222X','5223X','52241','52249','52291','52292','52299','53XXX']),'ISC_PARENT'] = "08. Transportation and Storage"
B002.loc[B002.M_Msic_Code.isin(['55101A','55101B','55101C','55101D','55104','55105','55108','55199','5520X','55900','56101','56102','56103','56104','56105','56106','56107','562XX','56302','56303','56399A','56399B']),'ISC_PARENT'] = "09. Accommodation and Food Service Activities"
B002.loc[B002.M_Msic_Code.isin(['58110','5812X','5813X','5819X','582XX','59XXX','60XXX','6110X','6120X','6130X','6190X','620XX','631XX','6391X','63990']),'ISC_PARENT'] = "10. Information and Communication"
B002.loc[B002.M_Msic_Code.isin(['64110','64191','64192','64193','64194','64195','64199','64200','64301','64302','64303','64304','64309','64910','64921','64922','64923','64924','64925','64929','64991','64992','64993','64999','651XX','6520X','65301','65302','66111','66112','66113','66114','66119','66121','66122','66123','66124','66125','66129','66191','66192','66199','66211','66212','66221','66222','66223','66224','66290','66301','66302','66303']),'ISC_PARENT'] = "11. Financial and Insurance/ Takaful Activities"
B002.loc[B002.M_Msic_Code.isin(['68101A','68101B','68101C','68102A','68102B','68102C','68102D','68102E','68103A','68103B','68104A','68104B','68104C','68104D','68104E','68104F','68104G','68109','68201','68202','68203','68209']),'ISC_PARENT'] =  "12. Real Estate Activities"
B002.loc[B002.M_Msic_Code.isin(['69XXX','70100','70201','70202','70203','70209','7110X','71200','72102','72103','72105','72106','72199','7220X','73100','73200','74XXX','75000']),'ISC_PARENT'] = "13. Professional, Scientific and Technical Activities"
B002.loc[B002.M_Msic_Code.isin(['7710X','772XX','77301','77307','77399','77400','78XXX','79XXX','80XXX','81XXX','821XX','82200','8230X','82910','8292XA','8292XB','8292XC','82990']),'ISC_PARENT'] = "14. Administrative and Support Service Activities"
B002.loc[B002.M_Msic_Code.isin(['841XX','842XX','8430X']),'ISC_PARENT'] = "15. Public Administration and Defence; Compulsory Social Security"
B002.loc[B002.M_Msic_Code.isin(['8510X','852XX','8530X','854XX','8550X']),'ISC_PARENT'] = "16. Education"
B002.loc[B002.M_Msic_Code.isin(['861XX','8620X','8690X','87XXX','88XXX']),'ISC_PARENT'] = "17. Human Health and Social Work Activities"
B002.loc[B002.M_Msic_Code.isin(['9000X','910XX','92000','93XXX']),'ISC_PARENT'] = "18. Arts, Entertainment and Recreation"
B002.loc[B002.M_Msic_Code.isin(['94XXXA','94XXXB','95XXX','960XX']),'ISC_PARENT'] = "19. Other Service Activities"
B002.loc[B002.M_Msic_Code.isin(['TXXXX']),'ISC_PARENT'] = "20. Activities of Households as Employers; Undifferentiated Goods and Services Producing Activities of Households for Own Use";
B002.loc[B002.M_Msic_Code.isin(['99000']),'ISC_PARENT'] = "21. Activities of Extraterritorial Organisations and Bodies"
B002.loc[B002.M_Msic_Code.isin(['97000']),'ISC_PARENT'] = "22. Household"
B002.loc[B002.ISC_PARENT.isna(),'ISC_PARENT'] = "23. Not Populated"


B002.Nature_Of_Business = B002.Nature_Of_Business.astype(str)

def SECTOR_GROUP(Nature_Of_Business):
    if Nature_Of_Business=='9999':
        return '9999'
    elif Nature_Of_Business[0] == '1': 
        return '1000'
    elif Nature_Of_Business[0] == '2': 
        return '2000'
    elif Nature_Of_Business[0] == '3': 
        return '3000'
    elif Nature_Of_Business[0] == '4': 
        return '4000'
    elif Nature_Of_Business[0] == '5': 
        return '5000'
    elif Nature_Of_Business[0] == '6': 
        return '6000'
    elif Nature_Of_Business[0] == '7': 
        return '7000'
    elif Nature_Of_Business[0] == '8': 
        return '8000'
    elif Nature_Of_Business[0] == '9': 
        return '9000' 
    else: 
        return '9999'
B002['SECTOR_GROUP'] = B002.apply(lambda x: SECTOR_GROUP(x['Nature_Of_Business']), axis=1)

def SECTOR_GROUP_DESC(Nature_Of_Business):
    if Nature_Of_Business=='9999':
        return 'SECTOR N.E.C'
    elif Nature_Of_Business[0] == '1': 
        return 'PRIMARY AGRICULTURE'
    elif Nature_Of_Business[0] == '2': 
        return 'MINING AND QUARRYING'
    elif Nature_Of_Business[0] == '3': 
        return 'MANUFACTURING (INCLUDING AGRO-BASED)'
    elif Nature_Of_Business[0] == '4': 
        return 'ELECTRICITY, GAS AND WATER SUPPLY'
    elif Nature_Of_Business[0] == '5': 
        return 'CONSTRUCTION'
    elif Nature_Of_Business[0] == '6': 
        return 'WHOLESALE AND RETAIL TRADE, AND RESTAURANT AND HOTELS'
    elif Nature_Of_Business[0] == '7': 
        return 'TRANSPORT, STORAGE AND COMMUNICATION'
    elif Nature_Of_Business[0] == '8': 
        return 'FINANCE, REAL ESTATE AND BUSINESS ACTIVITIES'
    elif Nature_Of_Business[0] == '9': 
        return 'EDUCATION, HEALTH AND OTHERS'
    else: 
        return 'SECTOR N.E.C'
B002['SECTOR_GROUP_DESC'] = B002.apply(lambda x: SECTOR_GROUP_DESC(x['Nature_Of_Business']), axis=1)

B003 = B002.merge(RATING_CATEGORY[['Combined_Rating','Risk_Category']].rename(columns = {'Combined_Rating':'COMBINED_RATING'}), on='COMBINED_RATING', how='left')

B000 = B003.sort_values(by=['M_Account_No'],ascending=[True])
B000['ACCOUNT_NO'] = B000.M_Account_No.str.strip()

#/*REFERENCE MARKET SEGMENT & GROUP*/
B004A = B003.merge(SUB_MARKET_SEGMENT.rename(columns = {'Code':'M_Sub_Market_Segment_Cd','Description':'M_Sub_Market_Segment_DESC'}), on='M_Sub_Market_Segment_Cd', how='left')

#/*REFERENCE CORP LIST & SCORECARD*/
B004 = B004A.merge(CRRS[['GCIF_NUM','SCORECARDS']].rename(columns = {'GCIF_NUM':'M_Cus_No'}), on='M_Cus_No', how='left')

MIB_CF = B004

MIB_CF.COMBINED_RATING= MIB_CF.COMBINED_RATING.str.upper()

MIB_CF.loc[MIB_CF.COMBINED_RATING.isin(["UNRATED"]), 'Risk_Category']="UNRATED"
MIB_CF.loc[MIB_CF.COMBINED_RATING.isin(["DEFAULTED"]), 'Risk_Category']="Sub-standard"
#/*  MANUAL ADJUSTMENT*/
MIB_CF.loc[MIB_CF.COMBINED_RATING.isin(["BKNM1"]), 'Risk_Category']="UNRATED"

MIB_CF.loc[MIB_CF.FIN_FRR.isin(["A+","A"]), 'COLLATERAL_TYPE'] ="01.SECURED"
MIB_CF.loc[MIB_CF.FIN_FRR.isin(["B","C","D","E","F"]), 'COLLATERAL_TYPE'] ="02.PARTIALLY SECURED"
MIB_CF.loc[MIB_CF.FIN_FRR.isin(["G","NA","?",""]), 'COLLATERAL_TYPE'] ="03.CLEAN"
MIB_CF.loc[MIB_CF.FIN_FRR.isna(), 'COLLATERAL_TYPE']="04.NOT POPULATED"

MIB_CF.loc[MIB_CF.Risk_Category.isin(["Very Low"]), 'RISK_CAT']="01.VERY LOW RISK"
MIB_CF.loc[MIB_CF.Risk_Category.isin(["Low"]), 'RISK_CAT']="02.LOW RISK"
MIB_CF.loc[MIB_CF.Risk_Category.isin(["Moderate"]), 'RISK_CAT']="03.MODERATE RISK"
MIB_CF.loc[MIB_CF.Risk_Category.isin(["High"]), 'RISK_CAT']="04.HIGH RISK"
MIB_CF.loc[MIB_CF.Risk_Category.isin(["Sub-standard"]), 'RISK_CAT']="05.SUB-STANDARD"
MIB_CF.loc[MIB_CF.Risk_Category.isin(["UNRATED"]), 'RISK_CAT']="06.UNRATED"
MIB_CF.loc[MIB_CF.Risk_Category.isin(["SOVEREIGN"]), 'RISK_CAT']="07.SOVEREIGN"
MIB_CF.loc[MIB_CF.Risk_Category.isin(["PSE"]), 'RISK_CAT']="08.PSE"
MIB_CF.loc[MIB_CF.RISK_CAT.isna(), 'RISK_CAT']="09.NOT POPULATED"

MIB_CF.loc[MIB_CF.M_Sub_Market_Segment_DESC.isin(["GWB-CORP GLC"]) , 'SEGMENT_TYPE']="01.GLC"
MIB_CF.loc[MIB_CF.M_Sub_Market_Segment_DESC.isin(["GWB-CORP MNC"]) , 'SEGMENT_TYPE']="02.MNC"
MIB_CF.loc[MIB_CF.M_Sub_Market_Segment_DESC.isin(["GWB-CORP MC"]) , 'SEGMENT_TYPE']="03.MC"
MIB_CF.loc[MIB_CF.M_Sub_Market_Segment_DESC.isin(["GWB-INTERNATIONAL"]) , 'SEGMENT_TYPE']="04.INTERNATIONAL"
MIB_CF.loc[MIB_CF.SEGMENT_TYPE.isna(),'SEGMENT_TYPE'] = "05.NOT POPULATED"


#/*TO CREATE BORROWER LEVEL DATASET*/
MIB_CF.M_Bnm_Balance_SUM_1.fillna(0,inplace=True)
CUS_BAL_TL = MIB_CF.iloc[np.where(MIB_CF.Facility.isin(["TL","TL_USD","TL_FC"]))][['CUS_NO','M_Bnm_Balance_SUM_1']].rename( columns = {'M_Bnm_Balance_SUM_1':'CUS_BAL_TL'}).groupby(['CUS_NO'])[['CUS_BAL_TL']].sum().reset_index()

CUS_BAL_TB = MIB_CF.iloc[np.where(MIB_CF.Facility.isin(["TB"]))][['CUS_NO','M_Bnm_Balance_SUM_1']].rename( columns = {'M_Bnm_Balance_SUM_1':'CUS_BAL_TB'}).groupby(['CUS_NO'])[['CUS_BAL_TB']].sum().reset_index()

CUS_BAL_OD = MIB_CF.iloc[np.where(MIB_CF.Facility.isin(["OD"]))][['CUS_NO','M_Bnm_Balance_SUM_1']].rename( columns = {'M_Bnm_Balance_SUM_1':'CUS_BAL_OD'}).groupby(['CUS_NO'])[['CUS_BAL_OD']].sum().reset_index()

CUS_BAL_STRC = MIB_CF.iloc[np.where(MIB_CF.Facility.isin(["STRC","STRC_USD","STRC_FC"]))][['CUS_NO','M_Bnm_Balance_SUM_1']].rename( columns = {'M_Bnm_Balance_SUM_1':'CUS_BAL_STRC'}).groupby(['CUS_NO'])[['CUS_BAL_STRC']].sum().reset_index()

#/*JUST ADDED AS AT 14 AUG 2018. ADDITIONAL INFO SINCE JUNE 2018 REPORT FOR RTO*/
CUS_BAL_RTO = MIB_CF.iloc[np.where(MIB_CF.Facility.isin(["RTO"]))][['CUS_NO','M_Bnm_Balance_SUM_1']].rename( columns = {'M_Bnm_Balance_SUM_1':'CUS_BAL_RTO'}).groupby(['CUS_NO'])[['CUS_BAL_RTO']].sum().reset_index()

row = ['M_Financing_Type_Cd','CUS_NO','Name','EWS_FLAG','WATCHLIST_FLAG','SMA_FLAG','IMPAIRED_FLAG','NEWLY_IF_FLAG','RESCHEDULED_FLAG','RESTRUCTURED_FLAG','R_N_R_FLAG','Nature_Of_Business','SECTOR_GROUP','SECTOR_GROUP_DESC','M_Msic_Code','ISC_PARENT','COMBINED_RATING','Risk_Category','RISK_CAT','SEGMENT_TYPE','FIN_FRR','COLLATERAL_TYPE','CUST_GROUP','SCORECARDS']

column = ['SMA_CUS_BAL','Authorised_Limit_SUM','M_Approved_Limit_SUM','M_Bnm_Balance_SUM_1']

MIB_CF.fillna(0,inplace=True)

MIB_CF_CUS_001 = MIB_CF.iloc[np.where(MIB_CF.M_Financing_Type_Cd=='I')].groupby(row)[column].sum().reset_index().rename( columns = {'SMA_CUS_BAL':'SMA_CUS_BAL','Authorised_Limit_SUM':'CUS_AUTHOR_LIMIT','M_Approved_Limit_SUM':'CUS_APPROV_LIMIT','M_Bnm_Balance_SUM_1':'CUS_BAL'})

MIB_CF_CUS_002 = MIB_CF_CUS_001.merge(CUS_BAL_TL, on='CUS_NO', how='left').merge(CUS_BAL_OD, on='CUS_NO', how='left').merge(CUS_BAL_TB, on='CUS_NO', how='left').merge(CUS_BAL_STRC, on='CUS_NO', how='left').merge(CUS_BAL_RTO, on='CUS_NO', how='left')


MIB_CF_CUS_002['CUS_LIMIT'] = MIB_CF_CUS_002.CUS_AUTHOR_LIMIT + MIB_CF_CUS_002.CUS_APPROV_LIMIT

MIB_CF_CUS_002.loc[MIB_CF_CUS_002.IMPAIRED_FLAG == 1,'CUS_STATUS'] = "IMPAIRED"
MIB_CF_CUS_002.loc[MIB_CF_CUS_002.SMA_FLAG == 1,'CUS_STATUS'] = "SMA"
MIB_CF_CUS_002.loc[MIB_CF_CUS_002.WATCHLIST_FLAG == 1,'CUS_STATUS'] = "WATCHLIST"
MIB_CF_CUS_002.loc[MIB_CF_CUS_002.EWS_FLAG == 1,'CUS_STATUS'] = "EWS"
MIB_CF_CUS_002.loc[MIB_CF_CUS_002.CUS_STATUS.isna(),'CUS_STATUS'] = "PERFORMING"

MIB_CF_CUS_002.loc[MIB_CF_CUS_002.RESCHEDULED_FLAG == 1,'RR_STATUS'] = "RESCHEDULED"
MIB_CF_CUS_002.loc[MIB_CF_CUS_002.RESTRUCTURED_FLAG == 1,'RR_STATUS'] = "RESTRUCTURED"
MIB_CF_CUS_002.loc[MIB_CF_CUS_002.RR_STATUS.isna(),'RR_STATUS'] = "N"

In [12]:
RDMS.COUNTERPARTY_ORIGINAL.fillna(0, inplace=True)
RDMS.EAD.fillna(0, inplace=True)
RDMS.RWA.fillna(0, inplace=True)
RDMS.EL.fillna(0, inplace=True)

RDMS_CUS_ALL = RDMS.groupby(['COUNTERPARTY_ORIGINAL'])[['EAD','RWA','EL']].sum().reset_index().rename(columns = {'COUNTERPARTY_ORIGINAL':'CUS_NO','EAD':'CUS_EAD','RWA':'CUS_RWA','EL':'CUS_EL'})

In [13]:
MIB_CF_CUS_002.CUS_NO = MIB_CF_CUS_002.CUS_NO.astype(str)
MIB_CF_CUS_002.CUS_NO = MIB_CF_CUS_002.CUS_NO.str.upper()
MIB_CF_CUS_002.CUS_NO = MIB_CF_CUS_002.CUS_NO.str.strip()

RDMS_CUS_ALL.CUS_NO = RDMS_CUS_ALL.CUS_NO.astype(str)
RDMS_CUS_ALL.CUS_NO = RDMS_CUS_ALL.CUS_NO.str.upper()
RDMS_CUS_ALL.CUS_NO = RDMS_CUS_ALL.CUS_NO.str.strip()

MIB_CF_CUS = MIB_CF_CUS_002.merge(RDMS_CUS_ALL, on='CUS_NO', how='left')

In [14]:
MIB_CF_CUS.CUS_BAL.fillna(0, inplace=True)

MIB_CF_CUS.loc[MIB_CF_CUS.CUS_BAL <= 1000000, 'FINANCING_SIZE_GROUP'] = "01.<=RM1M"
MIB_CF_CUS.loc[(MIB_CF_CUS.CUS_BAL > 1000000)    & (MIB_CF_CUS.CUS_BAL<= 3000000 ), 'FINANCING_SIZE_GROUP'] = "02.>RM1M <=RM3M"
MIB_CF_CUS.loc[(MIB_CF_CUS.CUS_BAL > 3000000)    & (MIB_CF_CUS.CUS_BAL<=5000000 ), 'FINANCING_SIZE_GROUP'] = "03.>RM3M <=RM5M"
MIB_CF_CUS.loc[(MIB_CF_CUS.CUS_BAL > 5000000)    & (MIB_CF_CUS.CUS_BAL<=10000000 ), 'FINANCING_SIZE_GROUP'] = "04.>RM5M <=RM10M"
MIB_CF_CUS.loc[(MIB_CF_CUS.CUS_BAL > 10000000)   & (MIB_CF_CUS.CUS_BAL<=15000000 ), 'FINANCING_SIZE_GROUP'] = "05.>RM10M <=RM15M"
MIB_CF_CUS.loc[(MIB_CF_CUS.CUS_BAL > 15000000)   & (MIB_CF_CUS.CUS_BAL<=30000000 ), 'FINANCING_SIZE_GROUP'] = "06.>RM15M <=RM30M"
MIB_CF_CUS.loc[(MIB_CF_CUS.CUS_BAL > 30000000)   & (MIB_CF_CUS.CUS_BAL<=50000000 ), 'FINANCING_SIZE_GROUP'] = "07.>RM30M <=RM50M"
MIB_CF_CUS.loc[(MIB_CF_CUS.CUS_BAL > 50000000)   & (MIB_CF_CUS.CUS_BAL<=100000000 ), 'FINANCING_SIZE_GROUP'] = "08.>RM50M <=RM100M"
MIB_CF_CUS.loc[(MIB_CF_CUS.CUS_BAL > 100000000)  & (MIB_CF_CUS.CUS_BAL<=500000000 ), 'FINANCING_SIZE_GROUP'] = "09.>RM100M <=RM500M"
MIB_CF_CUS.loc[(MIB_CF_CUS.CUS_BAL > 500000000)  & (MIB_CF_CUS.CUS_BAL<=1000000000 ), 'FINANCING_SIZE_GROUP'] = "10.>RM500M <=RM1000M"
MIB_CF_CUS.loc[(MIB_CF_CUS.CUS_BAL > 1000000000) & (MIB_CF_CUS.CUS_BAL<=1500000000 ), 'FINANCING_SIZE_GROUP'] = "11.>RM1000M <=RM1500M"
MIB_CF_CUS.loc[(MIB_CF_CUS.CUS_BAL > 1500000000) & (MIB_CF_CUS.CUS_BAL<=3000000000 ), 'FINANCING_SIZE_GROUP'] = "12.>RM1500M <=RM3000M"
MIB_CF_CUS.loc[MIB_CF_CUS.FINANCING_SIZE_GROUP.isna(), 'FINANCING_SIZE_GROUP'] ="13.>RM3000M"


In [15]:
#/*TO GET TOP30*/
TOP30_BORROWER = MIB_CF_CUS.sort_values(by=['CUS_BAL'],ascending=[False]).head(30)

TOP30_BORROWER['CUS_BAL_MIL'] = TOP30_BORROWER.CUS_BAL/1000000
TOP30_BORROWER['CUS_EAD_MIL'] = TOP30_BORROWER.CUS_EAD/1000000
TOP30_BORROWER['CUS_RWA_MIL'] = TOP30_BORROWER.CUS_RWA/1000000
TOP30_BORROWER['SEGMENT'] = TOP30_BORROWER.SEGMENT_TYPE.str[3:]
TOP30_BORROWER['TOP30IND'] = 1

TOP30_BORROWER1 = TOP30_BORROWER[['CUS_NO','Name','CUS_BAL_MIL','CUS_EAD_MIL','CUS_RWA_MIL','Risk_Category','COMBINED_RATING','FIN_FRR','SEGMENT','ISC_PARENT','TOP30IND']]

In [16]:
#/*TOP 20 GROUP*/

MIB_CF_CUS.CUST_GROUP.fillna('NA',inplace=True)
MIB_CF_CUS.CUS_BAL.fillna(0,inplace=True)

GROUP20_1 = MIB_CF_CUS.groupby(['CUST_GROUP'])[['CUS_BAL']].sum().reset_index()

GROUP20_1['CUS_BAL_MIL'] = GROUP20_1.CUS_BAL/1000000

GROUP20 = GROUP20_1.drop(['CUS_BAL'], axis = 1).sort_values(by=['CUS_BAL_MIL'], ascending=[False]).head(20)

GROUP20['TOP20GROUP'] = 1

In [17]:
#/*TOP 20 WATCHLIST*/
WATCHLIST_002 = MIB_CF_CUS.sort_values(by=['WATCHLIST_FLAG','CUS_BAL'],ascending=[False, False]).drop_duplicates('CUS_NO').head(20)

In [18]:
W001.columns = W001.columns.str.replace(" ", "_")

WATCHLIST_002['CUS_BAL_MIL'] = WATCHLIST_002.CUS_BAL/1000000

TOP20WATCHLIST = WATCHLIST_002[['CUS_NO','Name','CUS_BAL_MIL','COMBINED_RATING','FIN_FRR','SECTOR_GROUP_DESC','M_Msic_Code','ISC_PARENT']].merge(W001[['CUS_NO','REASON_FOR_INCLUSION_/STATUS']].drop_duplicates('CUS_NO',keep='last'), on='CUS_NO', how='left')

TOP20WATCHLIST['TOP20WL'] = 1

TOP20WATCHLIST.sort_values(by=['CUS_BAL_MIL'],ascending=[False],inplace=True)

In [21]:
#/*EWS*/

EWS_001 = MIB_CF_CUS.sort_values(by=['CUS_BAL'],ascending=[False])

EWS_001['EWS_FLAG'] = 1

EWS_001['CUS_BAL_MIL'] = EWS_001.CUS_BAL/1000000

EWS_001 = EWS_001[['CUS_NO','Name','CUS_BAL_MIL','COMBINED_RATING', 'FIN_FRR','ISC_PARENT']]


In [34]:
#/*SMA*/

B000.columns = B000.columns.str.strip()
B000.columns = B000.columns.str.upper()
 
S002.columns = S002.columns.str.strip()
S002.columns = S002.columns.str.upper()

B000.ACCOUNT_NO = B000.ACCOUNT_NO.str.strip()
B000.ACCOUNT_NO = B000.ACCOUNT_NO.str.upper()

S002.ACCOUNT_NO = S002.ACCOUNT_NO.str.strip()
S002.ACCOUNT_NO = S002.ACCOUNT_NO.str.upper()

SMA_001 = B000.merge(S002, on='ACCOUNT_NO', how='inner', suffixes=('', '_y')).drop_duplicates(keep='last')

SMA_001.M_BNM_BALANCE_SUM_1.fillna(0, inplace=True)

SMA_001['SMA_BAL'] = SMA_001.M_BNM_BALANCE_SUM_1/1000000

SMALIST_001 = SMA_001[['CUS_NO','ACCOUNT_NO','NAME','SMA_BAL','COMBINED_RATING','FIN_FRR','SECTOR_GROUP_DESC','M_MSIC_CODE','ISC_PARENT','TRIGGER_EVENTS_/_REMARKS']]


In [42]:
SMALIST_001.CUS_NO.fillna('NA',inplace=True)
SMALIST_001.NAME.fillna('NA',inplace=True)
SMALIST_001.COMBINED_RATING.fillna('NA',inplace=True)
SMALIST_001.FIN_FRR.fillna('NA',inplace=True)
SMALIST_001.SECTOR_GROUP_DESC.fillna('NA',inplace=True)
SMALIST_001.ISC_PARENT.fillna('NA',inplace=True)
SMALIST_001['TRIGGER_EVENTS_/_REMARKS'].fillna('NA',inplace=True)
SMALIST_001.SMA_BAL.fillna(0,inplace=True)

SMALIST  = SMALIST_001.groupby(['CUS_NO','NAME','COMBINED_RATING','FIN_FRR','SECTOR_GROUP_DESC','ISC_PARENT','TRIGGER_EVENTS_/_REMARKS'])[['SMA_BAL']].sum().reset_index()

SMALIST = SMALIST[['CUS_NO','NAME','SMA_BAL','COMBINED_RATING','FIN_FRR','SECTOR_GROUP_DESC','ISC_PARENT','TRIGGER_EVENTS_/_REMARKS']]

D:\anaconda3\lib\site-packages\pandas\core\generic.py:6245: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._update_inplace(new_data)


In [56]:
#/*IF*/

MIB_CF_CUS.columns = MIB_CF_CUS.columns.str.strip()
MIB_CF_CUS.columns = MIB_CF_CUS.columns.str.upper()

MIB_CF_CUS.IMPAIRED_FLAG.fillna(0,inplace=True)
MIB_CF_CUS.CUS_BAL.fillna(0,inplace=True)

IF_001 = MIB_CF_CUS.sort_values(by=['IMPAIRED_FLAG','CUS_BAL'],ascending=[False,False])

I001.OS_After_EIRUWI.fillna(0, inplace=True)

I001.CUS_NO = I001.CUS_NO.astype(str)
I001.CUS_NO = I001.CUS_NO.str.strip()
I001.CUS_NO = I001.CUS_NO.str.upper()

IF_001.CUS_NO = IF_001.CUS_NO.astype(str)
IF_001.CUS_NO = IF_001.CUS_NO.str.strip()
IF_001.CUS_NO = IF_001.CUS_NO.str.upper()

IFLIST = IF_001.merge(I001[['CUS_NO','OS_After_EIRUWI']],on='CUS_NO',how='left')

IFLIST['OS_AFTER_EIRUWI'] = IFLIST.OS_After_EIRUWI/1000000
IFLIST['CUS_BAL_MIL'] = IFLIST.CUS_BAL/1000000

IFLIST = IFLIST.groupby(['CUS_NO','NAME','CUS_BAL_MIL','FIN_FRR','COMBINED_RATING','COLLATERAL_TYPE', 'SEGMENT_TYPE', 'SECTOR_GROUP_DESC','M_MSIC_CODE', 'ISC_PARENT','IMPAIRED_FLAG', 'NEWLY_IF_FLAG'])[['OS_AFTER_EIRUWI']].sum().reset_index()

IFLIST = IFLIST[['CUS_NO','NAME','CUS_BAL_MIL','OS_AFTER_EIRUWI','FIN_FRR','COMBINED_RATING','COLLATERAL_TYPE','SEGMENT_TYPE','SECTOR_GROUP_DESC','M_MSIC_CODE','ISC_PARENT','IMPAIRED_FLAG','NEWLY_IF_FLAG']].sort_values(by=['OS_AFTER_EIRUWI'],ascending=[False])


In [68]:
#/*RESCHEDULED*/

RESCHEDULED_001 = MIB_CF_CUS.sort_values(by=['RESCHEDULED_FLAG','CUS_BAL'],ascending=[False,False])

RR001.columns = RR001.columns.str.replace(" ", "_")
RR001.columns = RR001.columns.str.strip()
RR001.columns = RR001.columns.str.upper()

RR001_CUS = RR001.rename(columns= {'AQ_QUALITY_STATUS':'STATUS_RR',\
                                   'CREDIT_FACILITY_LIMIT_MIL':'RR_CUS_LIMIT',\
                                   'CREDIT_FACILITY_O_S_MIL':'RR_CUS_BAL'}).\
groupby(['CUS_NO','NAME_OF_BORROWER','FREQUENCY','STATUS','REMARKS','STATUS_RR'])\
[['RR_CUS_LIMIT','RR_CUS_BAL']].sum().reset_index()

RESCHEDULED_001['CUS_BAL_MIL'] = RESCHEDULED_001.CUS_BAL/1000000

RESCHEDULED_001.CUS_NO = RESCHEDULED_001.CUS_NO.astype(str)
RESCHEDULED_001.CUS_NO = RESCHEDULED_001.CUS_NO.str.strip()
RESCHEDULED_001.CUS_NO = RESCHEDULED_001.CUS_NO.str.upper()

RR001_CUS.CUS_NO = RR001_CUS.CUS_NO.astype(str)
RR001_CUS.CUS_NO = RR001_CUS.CUS_NO.str.strip()
RR001_CUS.CUS_NO = RR001_CUS.CUS_NO.str.upper()

RESCHEDULEDLIST = RESCHEDULED_001[['CUS_NO','NAME','CUS_BAL_MIL','COMBINED_RATING','FIN_FRR']].merge(RR001_CUS[['CUS_NO','FREQUENCY','RR_CUS_LIMIT','RR_CUS_BAL','STATUS','REMARKS','STATUS_RR']], on='CUS_NO', how='left').sort_values(by=['RR_CUS_BAL'],ascending=[False])

RESCHEDULEDLIST.drop('CUS_NO', axis=1, inplace=True)

In [73]:
#/*RESTRUCTURED*/

RESTRUCTURED_001 = MIB_CF_CUS.sort_values(by=['RESTRUCTURED_FLAG','CUS_BAL'],ascending=[False,False])

RESTRUCTURED_001.CUS_NO = RESTRUCTURED_001.CUS_NO.astype(str)
RESTRUCTURED_001.CUS_NO = RESTRUCTURED_001.CUS_NO.str.strip()
RESTRUCTURED_001.CUS_NO = RESTRUCTURED_001.CUS_NO.str.upper()

RESTRUCTUREDLIST = RESTRUCTURED_001.merge(RR001_CUS, on='CUS_NO', how='left')

RESTRUCTUREDLIST['CUS_BAL_MIL'] = RESCHEDULED_001.CUS_BAL/1000000

RESTRUCTUREDLIST = RESTRUCTUREDLIST[['NAME','CUS_BAL_MIL','COMBINED_RATING','FIN_FRR','FREQUENCY','RR_CUS_LIMIT','RR_CUS_BAL']].sort_values(by=['RR_CUS_BAL'],ascending=[False])


In [75]:
#/*CUSBASE - JUST ADDED DEC 2018*/

MIB_CF_CUS1 = MIB_CF_CUS.merge(TOP30_BORROWER[['CUS_NO','TOP30IND']], on='CUS_NO', how='left').merge(TOP20WATCHLIST[['CUS_NO','TOP20WL']], on='CUS_NO', how='left').sort_values(by=['CUS_BAL'], ascending=[False])


In [81]:
#/*MASTER LISTING*/

MASTER_LIST = MIB_CF_CUS1.sort_values(by=['CUS_NO'],ascending=[True])

MASTER_LIST['CUS_LIMIT_MIL'] = MASTER_LIST.CUS_LIMIT/1000000 
MASTER_LIST['CUS_BAL_TL_MIL'] = MASTER_LIST.CUS_BAL_TL/1000000 
MASTER_LIST['CUS_BAL_TL_OD'] = MASTER_LIST.CUS_BAL_OD/1000000 
MASTER_LIST['CUS_BAL_STRC_MIL'] = MASTER_LIST.CUS_BAL_STRC/1000000 
MASTER_LIST['CUS_BAL_TB_MIL'] = MASTER_LIST.CUS_BAL_TB/1000000 
MASTER_LIST['CUS_BAL_RTO_MIL'] = MASTER_LIST.CUS_BAL_RTO/1000000 
MASTER_LIST['CUS_BAL_MIL'] = MASTER_LIST.CUS_BAL/1000000 
MASTER_LIST['SEGMENT'] = MASTER_LIST.SEGMENT_TYPE.str[3:]

MASTER_LIST = MASTER_LIST[['CUS_NO','NAME','CUS_LIMIT_MIL','CUS_BAL_TL_MIL','CUS_BAL_TL_OD','CUS_BAL_STRC_MIL','CUS_BAL_TB_MIL','CUS_BAL_RTO_MIL','CUS_BAL_MIL','RISK_CATEGORY','COMBINED_RATING','SECTOR_GROUP','NATURE_OF_BUSINESS','M_MSIC_CODE','ISC_PARENT','SEGMENT','CUS_STATUS','RR_STATUS','CUST_GROUP','SCORECARDS']]

In [85]:
#/*TO CREATE SUMMARY*/

MIB_CF.columns = MIB_CF.columns.str.strip()
MIB_CF.columns = MIB_CF.columns.str.upper()

SUMM_001 = MIB_CF[['RISK_CAT','M_BNM_BALANCE_SUM_1','CUS_NO']]


SUMM_001['BAL_MIL'] = SUMM_001.M_BNM_BALANCE_SUM_1/1000000
SUMM_001['NOC'] = 1

SUMM_001 = SUMM_001.groupby(['RISK_CAT'])[['BAL_MIL','NOC']].sum().reset_index().rename(columns = {'RISK_CAT':'ITEM_DESC_FLAG'})

SUMM_001['ITEM_FLAG'] = "01.PORTFOLIO BY RISK CATEGORY"

SUMM_001 = SUMM_001[['ITEM_FLAG','ITEM_DESC_FLAG','BAL_MIL','NOC']]

D:\anaconda3\lib\site-packages\ipykernel_launcher.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  if __name__ == '__main__':
D:\anaconda3\lib\site-packages\ipykernel_launcher.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  # Remove the CWD from sys.path while we load stuff.


In [87]:
SUMM_002 = MIB_CF[['COLLATERAL_TYPE','M_BNM_BALANCE_SUM_1','CUS_NO']]


SUMM_002['BAL_MIL'] = SUMM_002.M_BNM_BALANCE_SUM_1/1000000
SUMM_002['NOC'] = 1

SUMM_002 = SUMM_002.groupby(['COLLATERAL_TYPE'])[['BAL_MIL','NOC']].sum().reset_index().rename(columns = {'COLLATERAL_TYPE':'ITEM_DESC_FLAG'})

SUMM_002['ITEM_FLAG'] = "02.PORTFOLIO BY FRR CATEGORY"

SUMM_002 = SUMM_002[['ITEM_FLAG','ITEM_DESC_FLAG','BAL_MIL','NOC']]


D:\anaconda3\lib\site-packages\ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.
D:\anaconda3\lib\site-packages\ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """


In [91]:
SUMM_003 = MIB_CF[['SEGMENT_TYPE','M_BNM_BALANCE_SUM_1','CUS_NO']]


SUMM_003['BAL_MIL'] = SUMM_003.M_BNM_BALANCE_SUM_1/1000000
SUMM_003['NOC'] = 1

SUMM_003 = SUMM_003.groupby(['SEGMENT_TYPE'])[['BAL_MIL','NOC']].sum().reset_index().rename(columns = {'SEGMENT_TYPE':'ITEM_DESC_FLAG'})

SUMM_003['ITEM_FLAG'] = "03.PORTFOLIO BY SEGMENT"

SUMM_003 = SUMM_003[['ITEM_FLAG','ITEM_DESC_FLAG','BAL_MIL','NOC']]


D:\anaconda3\lib\site-packages\ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.
D:\anaconda3\lib\site-packages\ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """


In [93]:
SUMM_004 = MIB_CF[['SECTOR_GROUP','M_BNM_BALANCE_SUM_1','CUS_NO']]


SUMM_004['BAL_MIL'] = SUMM_004.M_BNM_BALANCE_SUM_1/1000000
SUMM_004['NOC'] = 1

SUMM_004 = SUMM_004.groupby(['SECTOR_GROUP'])[['BAL_MIL','NOC']].sum().reset_index().rename(columns = {'SECTOR_GROUP':'ITEM_DESC_FLAG'})

SUMM_004['ITEM_FLAG'] = "04.PORTFOLIO BY SECTOR"

SUMM_004 = SUMM_004[['ITEM_FLAG','ITEM_DESC_FLAG','BAL_MIL','NOC']]


D:\anaconda3\lib\site-packages\ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.
D:\anaconda3\lib\site-packages\ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """


In [95]:
SUMM_005 = MIB_CF[['ISC_PARENT','M_BNM_BALANCE_SUM_1','CUS_NO']]


SUMM_005['BAL_MIL'] = SUMM_005.M_BNM_BALANCE_SUM_1/1000000
SUMM_005['NOC'] = 1

SUMM_005 = SUMM_005.groupby(['ISC_PARENT'])[['BAL_MIL','NOC']].sum().reset_index().rename(columns = {'ISC_PARENT':'ITEM_DESC_FLAG'})

SUMM_005['ITEM_FLAG'] = "05.PORTFOLIO BY ISC PARENT"

SUMM_005 = SUMM_005[['ITEM_FLAG','ITEM_DESC_FLAG','BAL_MIL','NOC']]



D:\anaconda3\lib\site-packages\ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.
D:\anaconda3\lib\site-packages\ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """


In [100]:
SUMM_006 = MIB_CF_CUS[['FINANCING_SIZE_GROUP','CUS_BAL','CUS_NO']]


SUMM_006['BAL_MIL'] = SUMM_006.CUS_BAL/1000000
SUMM_006['NOC'] = 1

SUMM_006 = SUMM_006.groupby(['FINANCING_SIZE_GROUP'])[['BAL_MIL','NOC']].sum().reset_index().rename(columns = {'FINANCING_SIZE_GROUP':'ITEM_DESC_FLAG'})

SUMM_006['ITEM_FLAG'] = "06.PORTFOLIO BY FINANCING SIZE"

SUMM_006 = SUMM_006[['ITEM_FLAG','ITEM_DESC_FLAG','BAL_MIL','NOC']]


D:\anaconda3\lib\site-packages\ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.
D:\anaconda3\lib\site-packages\ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """


In [103]:
SUMM_007 = IFLIST[['COLLATERAL_TYPE','OS_AFTER_EIRUWI','CUS_NO']]


SUMM_007['BAL_MIL'] = SUMM_007.OS_AFTER_EIRUWI
SUMM_007['NOC'] = 1

SUMM_007 = SUMM_007.groupby(['COLLATERAL_TYPE'])[['BAL_MIL','NOC']].sum().reset_index().rename(columns = {'COLLATERAL_TYPE':'ITEM_DESC_FLAG'})

SUMM_007['ITEM_FLAG'] = "07.IF BY FRR CATEGORY"

SUMM_007 = SUMM_007[['ITEM_FLAG','ITEM_DESC_FLAG','BAL_MIL','NOC']]


D:\anaconda3\lib\site-packages\ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.
D:\anaconda3\lib\site-packages\ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """


In [105]:
SUMM_008 = IFLIST[['SEGMENT_TYPE','OS_AFTER_EIRUWI','CUS_NO']]


SUMM_008['BAL_MIL'] = SUMM_008.OS_AFTER_EIRUWI
SUMM_008['NOC'] = 1

SUMM_008 = SUMM_008.groupby(['SEGMENT_TYPE'])[['BAL_MIL','NOC']].sum().reset_index().rename(columns = {'SEGMENT_TYPE':'ITEM_DESC_FLAG'})

SUMM_008['ITEM_FLAG'] = "08.IF BY SEGMENT"

SUMM_008 = SUMM_008[['ITEM_FLAG','ITEM_DESC_FLAG','BAL_MIL','NOC']]


D:\anaconda3\lib\site-packages\ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.
D:\anaconda3\lib\site-packages\ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """


In [107]:
SUMM_009 = IFLIST[['SECTOR_GROUP_DESC','OS_AFTER_EIRUWI','CUS_NO']]


SUMM_009['BAL_MIL'] = SUMM_009.OS_AFTER_EIRUWI
SUMM_009['NOC'] = 1

SUMM_009 = SUMM_009.groupby(['SECTOR_GROUP_DESC'])[['BAL_MIL','NOC']].sum().reset_index().rename(columns = {'SECTOR_GROUP_DESC':'ITEM_DESC_FLAG'})

SUMM_009['ITEM_FLAG'] = "09.IF BY SECTOR"

SUMM_009 = SUMM_009[['ITEM_FLAG','ITEM_DESC_FLAG','BAL_MIL','NOC']]


D:\anaconda3\lib\site-packages\ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.
D:\anaconda3\lib\site-packages\ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """


In [109]:
SUMM_010 = IFLIST[['ISC_PARENT','OS_AFTER_EIRUWI','CUS_NO']]


SUMM_010['BAL_MIL'] = SUMM_010.OS_AFTER_EIRUWI
SUMM_010['NOC'] = 1

SUMM_010 = SUMM_010.groupby(['ISC_PARENT'])[['BAL_MIL','NOC']].sum().reset_index().rename(columns = {'ISC_PARENT':'ITEM_DESC_FLAG'})

SUMM_010['ITEM_FLAG'] = "10.IF BY ISC PARENT"

SUMM_010 = SUMM_010[['ITEM_FLAG','ITEM_DESC_FLAG','BAL_MIL','NOC']]


D:\anaconda3\lib\site-packages\ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.
D:\anaconda3\lib\site-packages\ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """


In [112]:
SUMM_011 = TOP30_BORROWER[['RISK_CAT','CUS_BAL','CUS_NO']]


SUMM_011['BAL_MIL'] = SUMM_011.CUS_BAL/1000000
SUMM_011['NOC'] = 1

SUMM_011 = SUMM_011.groupby(['RISK_CAT'])[['BAL_MIL','NOC']].sum().reset_index().rename(columns = {'RISK_CAT':'ITEM_DESC_FLAG'})

SUMM_011['ITEM_FLAG'] = "11.TOP30 BY RISK CATEGORY"

SUMM_011 = SUMM_011[['ITEM_FLAG','ITEM_DESC_FLAG','BAL_MIL','NOC']]


D:\anaconda3\lib\site-packages\ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.
D:\anaconda3\lib\site-packages\ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """


In [114]:
SUMM_012 = TOP30_BORROWER[['COLLATERAL_TYPE','CUS_BAL','CUS_NO']]


SUMM_012['BAL_MIL'] = SUMM_012.CUS_BAL/1000000
SUMM_012['NOC'] = 1

SUMM_012 = SUMM_012.groupby(['COLLATERAL_TYPE'])[['BAL_MIL','NOC']].sum().reset_index().rename(columns = {'COLLATERAL_TYPE':'ITEM_DESC_FLAG'})

SUMM_012['ITEM_FLAG'] = "12.TOP30 BY FRR CATEGORY"

SUMM_012 = SUMM_012[['ITEM_FLAG','ITEM_DESC_FLAG','BAL_MIL','NOC']]


D:\anaconda3\lib\site-packages\ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.
D:\anaconda3\lib\site-packages\ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """


In [116]:
SUMM_013 = MIB_CF_CUS.iloc[np.where(MIB_CF_CUS.EWS_FLAG==1)][['CUS_BAL','CUS_NO']]


SUMM_013['BAL_MIL'] = SUMM_013.CUS_BAL/1000000
SUMM_013['NOC'] = 1
SUMM_013['ITEM_DESC_FLAG'] = 'NA'

SUMM_013 = SUMM_013.groupby(['ITEM_DESC_FLAG'])[['BAL_MIL','NOC']].sum().reset_index().rename(columns = {'ITEM_DESC_FLAG':'ITEM_DESC_FLAG'})

SUMM_013['ITEM_FLAG'] = "13.EWS"

SUMM_013 = SUMM_013[['ITEM_FLAG','ITEM_DESC_FLAG','BAL_MIL','NOC']]


In [118]:
SUMM_014 = MIB_CF_CUS.iloc[np.where(MIB_CF_CUS.WATCHLIST_FLAG==1)][['CUS_BAL','CUS_NO']]


SUMM_014['BAL_MIL'] = SUMM_014.CUS_BAL/1000000
SUMM_014['NOC'] = 1
SUMM_014['ITEM_DESC_FLAG'] = 'NA'

SUMM_014 = SUMM_014.groupby(['ITEM_DESC_FLAG'])[['BAL_MIL','NOC']].sum().reset_index().rename(columns = {'ITEM_DESC_FLAG':'ITEM_DESC_FLAG'})

SUMM_014['ITEM_FLAG'] = "14.WATCHLIST"

SUMM_014 = SUMM_014[['ITEM_FLAG','ITEM_DESC_FLAG','BAL_MIL','NOC']]


In [121]:
SUMM_015 = SMALIST[['SMA_BAL','NAME']]


SUMM_015['BAL_MIL'] = SUMM_015.SMA_BAL
SUMM_015['NOC'] = 1
SUMM_015['ITEM_DESC_FLAG'] = 'NA'

SUMM_015 = SUMM_015.groupby(['ITEM_DESC_FLAG'])[['BAL_MIL','NOC']].sum().reset_index().rename(columns = {'ITEM_DESC_FLAG':'ITEM_DESC_FLAG'})

SUMM_015['ITEM_FLAG'] = "15.SMA"

SUMM_015 = SUMM_015[['ITEM_FLAG','ITEM_DESC_FLAG','BAL_MIL','NOC']]


D:\anaconda3\lib\site-packages\ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.
D:\anaconda3\lib\site-packages\ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """
D:\anaconda3\lib\site-packages\ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.o

In [123]:
SUMM_016 = IFLIST.iloc[np.where(IFLIST.NEWLY_IF_FLAG==1)][['OS_AFTER_EIRUWI','CUS_NO']]


SUMM_016['BAL_MIL'] = SUMM_016.OS_AFTER_EIRUWI
SUMM_016['NOC'] = 1
SUMM_016['ITEM_DESC_FLAG'] = 'NA'

SUMM_016 = SUMM_016.groupby(['ITEM_DESC_FLAG'])[['BAL_MIL','NOC']].sum().reset_index().rename(columns = {'ITEM_DESC_FLAG':'ITEM_DESC_FLAG'})

SUMM_016['ITEM_FLAG'] = "16.NEWLY CLASSIFIED IF"

SUMM_016 = SUMM_016[['ITEM_FLAG','ITEM_DESC_FLAG','BAL_MIL','NOC']]


In [125]:
SUMM_017 = IFLIST.iloc[np.where(IFLIST.IMPAIRED_FLAG==1)][['OS_AFTER_EIRUWI','CUS_NO']]


SUMM_017['BAL_MIL'] = SUMM_017.OS_AFTER_EIRUWI
SUMM_017['NOC'] = 1
SUMM_017['ITEM_DESC_FLAG'] = 'NA'

SUMM_017 = SUMM_017.groupby(['ITEM_DESC_FLAG'])[['BAL_MIL','NOC']].sum().reset_index().rename(columns = {'ITEM_DESC_FLAG':'ITEM_DESC_FLAG'})

SUMM_017['ITEM_FLAG'] = "17.IMPAIRED FINANCING"

SUMM_017 = SUMM_017[['ITEM_FLAG','ITEM_DESC_FLAG','BAL_MIL','NOC']]



In [127]:
SUMM_018 = RESTRUCTUREDLIST[['RR_CUS_BAL','NAME']]


SUMM_018['BAL_MIL'] = SUMM_018.RR_CUS_BAL
SUMM_018['NOC'] = 1
SUMM_018['ITEM_DESC_FLAG'] = 'NA'

SUMM_018 = SUMM_018.groupby(['ITEM_DESC_FLAG'])[['BAL_MIL','NOC']].sum().reset_index().rename(columns = {'ITEM_DESC_FLAG':'ITEM_DESC_FLAG'})

SUMM_018['ITEM_FLAG'] = "18.RESTRUCTURED" 

SUMM_018 = SUMM_018[['ITEM_FLAG','ITEM_DESC_FLAG','BAL_MIL','NOC']]


D:\anaconda3\lib\site-packages\ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.
D:\anaconda3\lib\site-packages\ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """
D:\anaconda3\lib\site-packages\ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.o

In [129]:
SUMM_019 = RESCHEDULEDLIST[['RR_CUS_BAL','NAME']]


SUMM_019['BAL_MIL'] = SUMM_019.RR_CUS_BAL
SUMM_019['NOC'] = 1
SUMM_019['ITEM_DESC_FLAG'] = 'NA'

SUMM_019 = SUMM_019.groupby(['ITEM_DESC_FLAG'])[['BAL_MIL','NOC']].sum().reset_index().rename(columns = {'ITEM_DESC_FLAG':'ITEM_DESC_FLAG'})

SUMM_019['ITEM_FLAG'] = "19.RESCHEDULED" 

SUMM_019 = SUMM_019[['ITEM_FLAG','ITEM_DESC_FLAG','BAL_MIL','NOC']]



D:\anaconda3\lib\site-packages\ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.
D:\anaconda3\lib\site-packages\ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """
D:\anaconda3\lib\site-packages\ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.o

In [131]:
#/*SUMMARY*/

frames = [SUMM_001,SUMM_002,SUMM_003,SUMM_004,SUMM_005,SUMM_006,SUMM_007,SUMM_008,SUMM_009, SUMM_010,SUMM_011,SUMM_012,SUMM_013,SUMM_014,SUMM_015,SUMM_016,SUMM_017,SUMM_018,SUMM_019,] 

MIB_CB_BNM_SUMM = pd.concat(frames)

In [134]:
writer2 = pd.ExcelWriter(r"T:\MIB Risk Management\Credit Risk Analytics - Historical\02_Data_Source\01_Corporate_Financing\04_Dataset\\"+str(RDATE)+"\\MIB_CB_BNM_SUMM_01_"+str(RDATE)+"(python).xlsx",engine='xlsxwriter')

MIB_CB_BNM_SUMM.to_excel(writer2, sheet_name='SUMMARY', index = False)
MASTER_LIST.to_excel(writer2, sheet_name='MASTER', index = False)
TOP30_BORROWER.to_excel(writer2, sheet_name='TOP30', index = False)

EWS_001.to_excel(writer2, sheet_name='EWS', index = False)
TOP20WATCHLIST.to_excel(writer2, sheet_name='TOP20WATCHLIST', index = False)
SMALIST.to_excel(writer2, sheet_name='SMA', index = False)
IFLIST.to_excel(writer2, sheet_name='IF', index = False)
RESCHEDULEDLIST.to_excel(writer2, sheet_name='RESCHEDULED', index = False)
RESTRUCTUREDLIST.to_excel(writer2, sheet_name='RESTRUCTURED', index = False)
MIB_CF_CUS1.to_excel(writer2, sheet_name='CUSBASE', index = False)

writer2.save()